## NNLM (Neural Network Language Model)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

텍스트 데이터를 다룰 수 있는 언어모델 `NNLM`을 만들어 보자.

In [3]:
# NNLM : 단어를 임베딩하고 비선형 계층을 통해 다음 단어의 확률을 계산
class NNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, context_size, hidden_size):
        super(NNLM, self).__init__() # nn.Module 초기화
        self.embed = nn.Embedding(vocab_size, embed_size)  # 단어 임베딩 레이어 
        self.fc1 = nn.Linear(embed_size * context_size, hidden_size) # 첫 번째 완전 연결 레이어
        self.relu = nn.ReLU() # ReLU 활성화 함수
        self.fc2 = nn.Linear(hidden_size, vocab_size) # 두 번째 완전 연결 레이어
        self.log_softmax = nn.LogSoftmax(dim=1) # 로그 소프트맥스 함수

    def forward(self, x): # 순전파 함수
        embeds = self.embed(x).view((x.size(0), -1)) # 임베딩 후 평탄화
        output = self.fc1(embeds) # 첫 번째 완전 연결 레이어 통과
        output = self.relu(output) # ReLU 활성화 함수 적용
        output = self.fc2(output) # 두 번째 완전 연결 레이어 통과
        log_probs = self.log_softmax(output) # 로그 소프트맥스 적용
        return log_probs

In [ ]:
VOCAB_SIZE = 5000  # 어휘 사전 크기
EMBED_SIZE = 300   # 임베딩 차원
CONTEXT_SIZE = 4   # 문맥 크기 (이전 단어 수)
HIDDEN_SIZE = 128  # 은닉층 크기

# 모델 인스턴스 생성
model = NNLM(VOCAB_SIZE, EMBED_SIZE, CONTEXT_SIZE, HIDDEN_SIZE) 
model

NNLM(
  (embed): Embedding(5000, 300)
  (fc1): Linear(in_features=1200, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=5000, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)

In [ ]:
############# 샘플 입력 데이터 생성 #############

# 0부터 VOCAB_SIZE-1 사이의 정수로 채워진 텐서 생성, 크기 (8, CONTEXT_SIZE)
X = torch.randint(0, VOCAB_SIZE, (8, CONTEXT_SIZE))  

# 0부터 VOCAB_SIZE-1 사이의 정수로 채워진 텐서 생성, 크기 (8,)
y = torch.randint(0, VOCAB_SIZE, (8,))            

In [ ]:
############# 모델 학습 #############

# NLLLoss 손실 함수 사용
criterion = nn.NLLLoss() # 음의 로그 우도 손실 함수

# optim.Adam 옵티마이저 사용, 학습률 0.01
optimizer = optim.Adam(model.parameters(), lr=0.01) 

model.train()
optimizer.zero_grad() # 기울기 초기화
output = model(X) # 순전파
loss = criterion(output, y) # 손실 계산
loss.backward() # 역전파
optimizer.step() # 가중치 업데이트

print(f"Loss: {loss.item():.4f}")


Loss: 8.5580


[ 결론 ]  
- n-그램 : 확률 기반으로 다음단어를 예측
- n-그램 모델을 사용했던걸 신경망 모델로 옮기면서 일반화된 연산을 할 수 있게끔 만들어준거라서,  
어떤 형태와 구조를 가지고 있는지 알아두기 위한 실습.